### Make Predictions with k-nearest neighbors on the Iris Flowers Dataset

In [420]:
# Import library for the project
from csv import reader
from math import sqrt
import random

Open and load the csv

In [421]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

I split in 2 part the dataset (randomly), the first part is the training data and the second part will be use for the validation model
I will use cross validation in order to compare the diffrence

In [422]:
# split in 2 the dataset 
def splitcsv(dataset):
    validation_data = []
    trainning_data = []
    for i in range(0,len(dataset)):
        rd = random.choice([True, False])
        if rd:
            trainning_data.append(dataset[i]) 
        else:
            validation_data.append(dataset[i])
    return trainning_data, validation_data

    

Define function from scratch :

-Convert string column to float

-Convert string column to integer

-Find the min and max values for each column

-Rescale dataset columns to the range 0-1 (normalization)

-Calculate the Euclidean distance between two vectors

-Locate the most similar neighbors

In [423]:
def convert_str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
        # print('[%s] => %d' % (value, i))
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

def distance_euclid(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

def find_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = distance_euclid(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

Prediction
Make a prediction with neighbors

In [424]:
def prediction(train, test_row, num_neighbors):
    neighbors = find_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

Cross validation 

In [425]:
def cross_validation(dataset,k):
    # cross validation
    len_dataset = len(dataset)
    for i in range(k):
        # init dataset for training
        # init dataset for testing
        dataset_train = []
        dataset_test = []
        
        # process the split
        if i == (k-1):
            dataset_test = dataset[int(len_dataset/k)*(i):len_dataset]
            dataset_train = dataset[0:int(len_dataset/k)*i-1]

            print('K=%d partition n=%d '% (k,i))

            print('size of dataset test=%d'% (len(dataset_test)))
            print('[%d,%d]'% (int(len_dataset/k)*(i),len_dataset))

            print('size of dataset train= %d'% (len(dataset_train)))
            # print('[%d,%d]'% (0,int(len_dataset/k)*i-1))
            print('---')
            
        else:
            dataset_train = dataset.copy()
            dataset_test = dataset[int(len_dataset/k)*i:int(len_dataset/k)*(i+1)]
            del dataset_train[int(len_dataset/k)*i:int(len_dataset/k)*(i+1)]
           
            print('K=%d partition n=%d '% (k,i))

            print('size of dataset test=%d'% (len(dataset_test)))
            print('[%d,%d]'% (int(len_dataset/k)*i,int(len_dataset/k)*(i+1)))

            print('size of dataset train= %d'% (len(dataset_train)))

            print('---')
            
       
        # define model parameter
        num_neighbors = 4
        # define the value for check the ratio of true positive by class
        count_true = 0
        count_true_1 =0
        total_1 = 0
        count_true_2 = 0
        total_2 = 0
        count_true_3 = 0
        total_3 = 0
        # predict the label
        for j in range(0,len(dataset_test)):
            label = 0
            label = prediction(dataset_train, dataset_test[j][0:3], num_neighbors)
            # precision
            if dataset_test[j][4] == 0:
                    total_1 = total_1 + 1
            if dataset_test[j][4] == 1:
                    total_2 = total_2 + 1
            if dataset_test[j][4] == 2:
                    total_3 = total_3 + 1
            # validation 
            if dataset_test[j][4] == label:
                count_true = count_true + 1
                if label == 0:
                    count_true_1 = count_true_1 + 1
                elif label == 1:
                    count_true_2 = count_true_2 + 1
                elif label == 2:
                    count_true_3 = count_true_3 + 1
            
        pourcentage_tot = count_true/len(dataset_test)
        pourcentage_1 = count_true_1/total_1
        pourcentage_2 = count_true_2/total_2
        pourcentage_3 = count_true_3/total_3
        print('K=%d partition n=%d'% (k,i))
        print('there are %d true over %d prediction ' % (count_true,len(dataset_test)))
        print('CLASS 0 , there are %s true over %s prediction ' % (count_true_1,total_1))
        print('CLASS 1 , there are %s true over %s prediction ' % (count_true_2,total_2))
        print('CLASS 2 , there are %s true over %s prediction ' % (count_true_3,total_3))
        print('Pourcentage CLASS 0 ==> %s' % (pourcentage_1 ))
        print('Pourcentage CLASS 1 ==> %s' % (pourcentage_2 ))
        print('Pourcentage CLASS 2 ==> %s' % (pourcentage_3 ))
        print('Pourcentage TOTAL ==> %s' % (pourcentage_tot ))
        print('-------------------------------------------------------')


Make a prediction with KNN on Iris Dataset prediction 

class 0 is Iris-setosa

class 1 is Iris-versicolor

class 2 is Iris-virginica

In [426]:
filename = 'iris.csv'
dataset = load_csv(filename)
dataset_shuffle =[]
for j in range(len(dataset[0])-1):
	convert_str_column_to_float(dataset, j)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
dataset_shuffle = random.sample(dataset, len(dataset))

for k in range(2,10):	
	cross_validation(dataset_shuffle,k)



ver 75 prediction 
CLASS 0 , there are 21 true over 25 prediction 
CLASS 1 , there are 27 true over 27 prediction 
CLASS 2 , there are 12 true over 23 prediction 
Pourcentage CLASS 0 ==> 0.84
Pourcentage CLASS 1 ==> 1.0
Pourcentage CLASS 2 ==> 0.5217391304347826
Pourcentage TOTAL ==> 0.8
-------------------------------------------------------
K=3 partition n=0 
size of dataset test=50
[0,50]
size of dataset train= 100
---
K=3 partition n=0
there are 36 true over 50 prediction 
CLASS 0 , there are 15 true over 17 prediction 
CLASS 1 , there are 12 true over 13 prediction 
CLASS 2 , there are 9 true over 20 prediction 
Pourcentage CLASS 0 ==> 0.8823529411764706
Pourcentage CLASS 1 ==> 0.9230769230769231
Pourcentage CLASS 2 ==> 0.45
Pourcentage TOTAL ==> 0.72
-------------------------------------------------------
K=3 partition n=1 
size of dataset test=50
[50,100]
size of dataset train= 100
---
K=3 partition n=1
there are 39 true over 50 prediction 
CLASS 0 , there are 11 true over 16 pr